# Model interpretation

This code translates a model (that is, a set of results produced by Mallet) into a human-interpretable form so that we can label and categorize the topics.

The Mallet output that we will use comes in three files. There's a document-topic matrix (```doctopics```), a list of keywords (```keys```), and some ```diagnostics``` on the topics.

In [3]:
import pandas as pd
import numpy as np
from collections import Counter
import xml.etree.ElementTree as ET

While we remove only a minimal list of stopwords before topic inference, there are lots of words that human readers find difficult to interpret; those get filtered out before we present the list to them. We need to load that list.

In [2]:
with open('functionwords.txt', encoding = 'utf-8') as f:
    lines = f.readlines()
    functionwords = [x.strip() for x in lines]

We also need metadata for the corpus used in modeling. For all the models we'll be using this is "corpus4," which underwent some pruning of e.g. nonfiction and collected works.

In [12]:
corpus = pd.read_csv('../metadata/corpus4.tsv', sep = '\t', low_memory = False)

For any given model, we start by constructing a dictionary of diagnostic statistics associated with topics.

To interpret these, consult [the Mallet documentation.](http://mallet.cs.umass.edu/diagnostics.php)

In [40]:
def extract_diag_stats(xmlfile):
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    
    topics = dict()
    alltokencount = 0
    
    for item in root.findall('topic'):
        t = item.attrib
        tnum = int(t['id'])
        topics[tnum] = dict()
        
        topics[tnum]['tokens'] = float(t['tokens'])
        alltokencount += topics[tnum]['tokens']
        
        topics[tnum]['document_entropy'] = t['document_entropy']
        topics[tnum]['coherence'] = t['coherence']
        topics[tnum]['word-length'] = t['word-length']
        topics[tnum]['rank_1_docs'] = t['rank_1_docs']
    
    for tnum, valuedict in topics.items():
        valuedict['tokenpct'] = round(100 * topics[tnum]['tokens'] / alltokencount, 3)
        
    print(alltokencount)
    
    return topics
        

Next we translate the keywords into a more human-interpretable form by deleting function words that are too general.

In [41]:
def translate_keys(keyfile, topicdict):
    outlines = []
    global functionwords
    
    with open(keyfile, encoding ='utf-8') as f:
        for line in f.readlines():
            tokens = line.strip().split()
            topicnum = int(tokens[0])
            alpha = tokens[1]
            words = tokens[2 : ]
            wordsneeded = []
            for w in words:
                if w in functionwords:
                    continue
                else:
                    wordsneeded.append(w)
                
                if len(wordsneeded) > 75:
                    break
            
            thistopic = topicdict[topicnum]
            
            outline = '"' + 'TOPIC ' + str(topicnum) + '\n'
            outline = outline + 'pct corpus = ' + str(thistopic['tokenpct']) + '%\n'
            outline = outline + 'doc entropy = ' + thistopic['document_entropy'] + '\n'
            outline = outline + 'word length = ' + thistopic['word-length'] + '\n'
            outline = outline + 'coherence = ' + thistopic['coherence'] + '\n'
            outline = outline + 'rank 1 docs = ' + thistopic['rank_1_docs'] + '\n"\t"'
            
            for wordfloor in range(0, 70, 10):
                outline = outline + ' '.join(wordsneeded[wordfloor : (wordfloor + 10)]) + '\n'
            
            outline = outline + '"\t'
            
            outlines.append(outline)
    
    return outlines

Finally, we define a pair of functions that get the doctopic matrix and return the n documents with highest proportion of each topic.

In [5]:
def get_doctopics(filename):
    chunks = dict()
    with open(filename, encoding = 'utf-8') as f:
        for line in f:
            fields = line.strip().split()
            chunkid = fields[1]
            docid = fields[1].split('_')[0]
        
            if docid not in chunks:
                chunks[docid] = []
                
            vector = np.array([float(x) for x in fields[2: ]])
            chunks[docid].append(vector)
            
    docs = dict()
    docsizes = dict()    # not actually used in current version I think
                        # early on I used number of chunks as a proxy
                        # but now using actual number of tokens, from metadata
    for docid, value in chunks.items():
        avgvector = np.mean(value, axis = 0)
        docs[docid] = avgvector
        docsizes[docid] = len(vector)
    
    return docs, docsizes

def docs2maxdocs(docs, howmany):
    topictuples = dict()
    
    for docid, vector in docs.items():
        for idx, fraction in enumerate(vector):
            if idx not in topictuples:
                topictuples[idx] = []
            topictuples[idx].append((fraction, docid))
    
    maxdocs = dict()
    
    for topic, tuples in topictuples.items():
        tuples.sort(reverse = True)
        maxdocs[topic] = [(round(x[0], 3), x[1]) for x in tuples[0: howmany]]
        
    return maxdocs

def get_maxdocs(filename, howmany):
    docs, docsizes = get_doctopics(filename)
    maxdocs = docs2maxdocs(docs, howmany)
    return maxdocs, docsizes
    

The function below combines all the functions above into a single recipe.

Note, in particular, that we instruct maxdocs to return the top 200 documents for each topic, and use all 200 to infer, for instance, date quartiles.

But we only individually list the top seven.

In [44]:
def interpret_model(modelnumber):
    
    global corpus, functionwords
    
    doctopicfile = "final/k" + str(modelnumber) + 'doctopics.txt'
    keyfile = "final/k" + str(modelnumber) + 'keys.txt'
    diagnosticfile = "final/k" + str(modelnumber) + 'diagnostics.xml'
    
    topicdict = extract_diag_stats(diagnosticfile)
    
    outlines = translate_keys(keyfile, topicdict)
    
    maxdocs, docsizes = get_maxdocs(doctopicfile, 200)   # get the top 200 books
                                    # and use all 200 to find the "biggest author"
                                    # and "date quartiles"
    finalout = []

    
    for idx, outline in enumerate(outlines):
        outline = outline + '"'
        docctr = 0
        authorproportions = Counter()
        firstpubs = []
        for fraction, doc in maxdocs[idx]:
            author = corpus.loc[corpus.docid == doc, 'hathi_author'].values[0]
            if pd.isnull(author):
                author = "Unknown Author"
            else:
                if len(author) > 30:
                    author = author[0: 30]
            title = corpus.loc[corpus.docid == doc, 'hathi_title'].values[0]
            if pd.isnull(title):
                title = "Unknown Title"
            else:
                title = title.replace('"', '')
                if len(title) > 33:
                    title = title[0 : 33]
            firstpub = corpus.loc[corpus.docid == doc, 'firstpub'].values[0]
            firstpubs.append(firstpub)
            docctr += 1
            if docctr <= 7:   # but only list the top seven individually
                outline = outline + str(fraction) + ' | ' + author + ' | ' + title + ' | ' + str(firstpub) + '\n'
            
            authorproportions[author] += fraction * float(corpus.loc[corpus.docid == doc, 'tokens'].values[0])
            
        outline = outline + '"\t'
        
        allsum = sum(authorproportions.values())
        biggestauth, biggestsum = authorproportions.most_common(1)[0]
        
        bigfrac = round(100 * biggestsum / topicdict[idx]['tokens'], 2)
        outline = outline + biggestauth + " = " + str(bigfrac) + '%\t'
        percent25, percent75 = np.percentile(firstpubs, [25, 75])
        outline = outline + str(int(percent25)) + '-' + str(int(percent75)) + '\n'
        finalout.append(outline)
    
    outfile = keyfile.replace('keys.txt', 'interpret.tsv')
    with open(outfile, mode = 'w', encoding = 'utf=8') as f:
        f.write('topicstats\tkeywords\ttopbooks\tbiggestauth\tdatequartiles\n')
        for o in finalout:
            f.write(o)

In [45]:
interpret_model(200)

1400787771.0


## Unrelated stuff below

After starting to examine some early topic models, I found a lot of nonfiction floated to the top, and used this notebook to identify and remove some of it.

In [93]:
def docstoremove(topiclist, modelnumber, alreadyremoved, alreadycleared):
    global corpus, functionwords
    
    doctopicfile = "k" + str(modelnumber) + 'doctopics.txt'
    
    maxdocs = get_maxdocs(doctopicfile, 25)
    
    for t in topiclist:
        print()
        print('TOPIC ', t)
        suspects = maxdocs[t]
        for fraction, doc in suspects:
            # print(doc)
            if doc in alreadyremoved or doc in alreadycleared:
                continue
            else:
                author = corpus.loc[corpus.docid == doc, 'hathi_author'].values[0]
                title = corpus.loc[corpus.docid == doc, 'hathi_title'].values[0]
                print(fraction, ' | ', author, ' | ', title)
                user = input('remove? ')
                if user == 'y':
                    alreadyremoved.add(doc)
                else:
                    alreadycleared.add(doc)
    
    return alreadyremoved, alreadycleared
  

In [78]:
alreadyremoved = set()
alreadycleared = set()
suspicious = [5, 18]

In [94]:
alreadyremoved, alreadycleared = docstoremove(suspicious, 100, alreadyremoved, alreadycleared)


TOPIC  5
0.361  |  Farrington, Frank  |  Efficiency in the business


remove?  y


0.283  |  Watson, Nancy Dingman  |  What is one?


remove?  n


0.249  |  Butler, Ellis Parker  |  Robinson Crusoe and thrift stamps


...

remove?  n


0.43  |  Loori, John Daido  |  Mountain record of Zen talks


remove?  n


0.43  |  Hagen, Steve  |  Buddhism, plain and simple


remove?  y


In [95]:
alreadyremoved

{'inu.30000082137286',
 'inu.30000120170752',
 'mdp.39015005630002',
 'mdp.39015008901053',
 'mdp.39015009304042',
 'mdp.39015011804856',
 'mdp.39015019194730',
 'mdp.39015028446519',
 'mdp.39015032936745',
 'mdp.39015048719093',
 'mdp.39015048853934',
 'mdp.39015059449317',
 'mdp.39015081835764',
 'njp.32101078173828',
 'nnc2.ark+=13960=t6nz9cv81',
 'nyp.33433020487124',
 'pst.000043376155',
 'uc1.b3937989',
 'uc1.b3955246',
 'uc1.b4395331',
 'uc1.b5023357',
 'uc2.ark+=13960=t71v5pj5d',
 'wu.89097453831'}

In [96]:
suspicious = [20, 22, 95]
alreadyremoved, alreadycleared = docstoremove(suspicious, 100, alreadyremoved, alreadycleared)


TOPIC  20
0.446  |  McNerney, Kathryn  |  American oak furniture


remove?  y


0.36  |  nan  |  Official guides


remove?  y


0.31  |  Hansen, Henny Harald  |  Costumes and styles


...

remove?  y


0.18  |  Hertz, Marguerite Rosenberg  |  Frequency tables for scoring Rorschach responses; code charts, normal and rare details, F+ and F- responses, popular responses, original responses


remove?  y


0.179  |  Beebe, William  |  The Arcturus adventure; an account of the New York zoological so


remove?  n


In [97]:
alreadyremoved

{'coo.31924003587999',
 'ien.35556030137137',
 'ien.35556030796155',
 'inu.30000082137286',
 'inu.30000120170752',
 'loc.ark+=13960=t7kp8vh4x',
 'mdp.35112203985603',
 'mdp.39015001116238',
 'mdp.39015005630002',
 'mdp.39015006310232',
 'mdp.39015007548012',
 'mdp.39015008901053',
 'mdp.39015009240063',
 'mdp.39015009304042',
 'mdp.39015009856363',
 'mdp.39015011804856',
 'mdp.39015014718244',
 'mdp.39015014846086',
 'mdp.39015019194730',
 'mdp.39015028446519',
 'mdp.39015031013892',
 'mdp.39015032936745',
 'mdp.39015033591671',
 'mdp.39015037792010',
 'mdp.39015048719093',
 'mdp.39015048853934',
 'mdp.39015059449317',
 'mdp.39015062453520',
 'mdp.39015066348353',
 'mdp.39015081835764',
 'mdp.39076005163303',
 'mdp.39076005640987',
 'mdp.49015002057033',
 'njp.32101078173828',
 'nnc2.ark+=13960=t6nz9cv81',
 'nyp.33433020487124',
 'nyp.33433066630348',
 'pst.000008276803',
 'pst.000010587188',
 'pst.000029752102',
 'pst.000043376155',
 'uc1.$b288145',
 'uc1.31822000601435',
 'uc1.321060

In [98]:
suspicious = [41]

In [99]:
alreadyremoved, alreadycleared = docstoremove(suspicious, 300, alreadyremoved, alreadycleared)


TOPIC  41
0.329  |  nan  |  POST course certification and presentation guidelines


remove?  y


0.212  |  nan  |  Course evaluations


remove?  y


0.21  |  nan  |  [SPEC kit on the systems function in ARL libraries


...

remove?  n


0.108  |  Boulle, Pierre  |  A noble profession


remove?  n


0.105  |  Kimbrough, Emily  |  Through Charley's door


remove?  n


In [101]:
len(alreadyremoved)

68

In [102]:
corpus.shape

(29611, 23)

In [103]:
corpus = corpus.loc[~corpus.docid.isin(alreadyremoved), : ]
corpus.shape

(29543, 23)

In [104]:
pwd

'/Users/tunder/Dropbox/python/cohort/modelselection'

In [105]:
corpus.to_csv('../metadata/corpus3.tsv', sep = '\t', index = False)

In [106]:
oldcorpus = pd.read_csv('../metadata/modelcorpus.tsv', sep = '\t')

/Users/tunder/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [107]:
len(corpus.loc[corpus.firstpub == 1998, : ])

230

In [108]:
len(oldcorpus.loc[(oldcorpus.firstpub == 1999) & (oldcorpus.nonficprob < .95), :])

299

In [120]:
supplement = oldcorpus.loc[(oldcorpus.firstpub == 1999) & (oldcorpus.nonficprob < .95), :].sample(225)

In [121]:
supplement.head()

,docid,hathi_author,hathi_title,authordate,birthyear,deathyear,chi_date,ukw_date,copyright_date,firstpub,datewrong,nationality,nonficprob,distances,copyright_corpus,manual_corpus,us_national,pubdate_known,authof3ormore
38922,uc1.32106016419647,"Jooste, Pamela",Frieda and Min,NaN,NaN,NaN,NaN,NaN,NaN,1999,NaN,NaN,0.172311,NaN,False,False,False,False,False
39064,mdp.39015048930781,"Nissen, Thisbe",Out of the girls' room and into the night,1972-,1972.0,NaN,NaN,NaN,NaN,1999,NaN,NaN,0.398568,NaN,False,False,False,False,True
38960,mdp.39015045985358,"Ellis, Trey","Right here, right now",NaN,NaN,NaN,NaN,NaN,NaN,1999,NaN,NaN,0.409676,NaN,False,False,False,False,False
39124,mdp.39015047609089,"Mayo, Wendell",B. Horror : and other stories,NaN,NaN,NaN,NaN,NaN,NaN,1999,NaN,NaN,0.363226,NaN,False,False,False,False,False
38953,mdp.39015048735289,"Staffel, Megan",The notebook of lost things,1952-,1952.0,NaN,NaN,NaN,NaN,1999,NaN,NaN,0.243718,NaN,False,False,False,False,False


In [110]:
set(corpus.columns) - set(supplement.columns)

{'pagesinchunk', 'skipped_pages', 'tokens', 'trimmed_pages'}

In [122]:
parsed = pd.read_csv('../getEF/parsing_metadata.tsv', sep = '\t')
parsed.head()

,id,tokens,skipped_pages,trimmed_pages,pagesinchunk
0,nyp.33433082289871_0,10150,0,36,75
1,nyp.33433082289871_1,9632,0,36,73
2,nyp.33433075765176_0,6921,0,61,62
3,nyp.33433075765176_1,6884,0,61,62
4,nyp.33433075765176_2,6927,0,61,62


In [123]:
def get_docid(astring):
    return astring.split('_')[0]

parsed = parsed.assign(docid = parsed['id'].apply(get_docid))

In [127]:
docsums = parsed.groupby('docid').sum()
docsums.reset_index(inplace = True)
docsums.head()

,docid,tokens,skipped_pages,trimmed_pages,pagesinchunk
0,aeu.ark+=13960=t22c08w6m,46847,0,248,256
1,aeu.ark+=13960=t2p575j24,50724,0,260,266
2,aeu.ark+=13960=t3126wn1n,17734,0,58,122
3,aeu.ark+=13960=t3gx5k62v,40509,0,264,269
4,aeu.ark+=13960=t4qj90j8c,32848,0,304,305


In [128]:
supplement = supplement.merge(docsums.loc[:, ['docid', 'pagesinchunk', 'tokens']], on= 'docid')

In [129]:
supplement.head()

,docid,hathi_author,hathi_title,authordate,birthyear,deathyear,chi_date,ukw_date,copyright_date,firstpub,...,nationality,nonficprob,distances,copyright_corpus,manual_corpus,us_national,pubdate_known,authof3ormore,pagesinchunk,tokens
0,uc1.32106016419647,"Jooste, Pamela",Frieda and Min,NaN,NaN,NaN,NaN,NaN,NaN,1999,...,NaN,0.172311,NaN,False,False,False,False,False,286,57387
1,mdp.39015048930781,"Nissen, Thisbe",Out of the girls' room and into the night,1972-,1972.0,NaN,NaN,NaN,NaN,1999,...,NaN,0.398568,NaN,False,False,False,False,True,217,52567
2,mdp.39015045985358,"Ellis, Trey","Right here, right now",NaN,NaN,NaN,NaN,NaN,NaN,1999,...,NaN,0.409676,NaN,False,False,False,False,False,238,42974
3,mdp.39015047609089,"Mayo, Wendell",B. Horror : and other stories,NaN,NaN,NaN,NaN,NaN,NaN,1999,...,NaN,0.363226,NaN,False,False,False,False,False,123,28058
4,mdp.39015048735289,"Staffel, Megan",The notebook of lost things,1952-,1952.0,NaN,NaN,NaN,NaN,1999,...,NaN,0.243718,NaN,False,False,False,False,False,198,36106


In [133]:
corpus = pd.read_csv('../metadata/corpus3.tsv', sep = '\t', low_memory = False)
corpus.drop(columns = ['skipped_pages', 'trimmed_pages'], inplace = True)

In [134]:
corpus.shape

(29482, 21)

In [135]:
corpus = pd.concat([corpus, supplement])
corpus.shape

(29707, 21)

In [138]:
corpus.to_csv('../metadata/corpus4.tsv',sep = '\t', index = False)

In [137]:
corpus = corpus.assign(tokensperpage = np.round(corpus.tokens / corpus.pagesinchunk, 3))

In [139]:
allowed = corpus.docid.tolist()
len(allowed)

29707

In [141]:
paths = pd.read_csv('../getEF/pathlistwithauthors.tsv', sep = '\t')
subpaths = paths.loc[paths.docid.isin(allowed), : ]
subpaths.shape

(29707, 3)

In [142]:
subpaths.head()

,docid,path,author
3,uc1.b4975632,uc1/pairtree_root/b4/97/56/32/b4975632/uc1.b49...,"Aakhus, Patricia"
4,inu.30000112046630,inu/pairtree_root/30/00/01/12/04/66/30/3000011...,"Abbe, George"
5,uc1.$b799882,uc1/pairtree_root/$b/79/98/82/$b799882/uc1.$b7...,"Abbe, George"
6,uc1.$b149331,uc1/pairtree_root/$b/14/93/31/$b149331/uc1.$b1...,"Abbe, George"
7,uc1.32106007981415,uc1/pairtree_root/32/10/60/07/98/14/15/3210600...,"Abbey, Edward"


In [145]:
subpaths = subpaths.merge(corpus.loc[: , ['docid', 'tokensperpage']], on = 'docid')
subpaths.head()

,docid,path,author,tokensperpage
0,uc1.b4975632,uc1/pairtree_root/b4/97/56/32/b4975632/uc1.b49...,"Aakhus, Patricia",189.321
1,inu.30000112046630,inu/pairtree_root/30/00/01/12/04/66/30/3000011...,"Abbe, George",191.688
2,uc1.$b799882,uc1/pairtree_root/$b/79/98/82/$b799882/uc1.$b7...,"Abbe, George",211.281
3,uc1.$b149331,uc1/pairtree_root/$b/14/93/31/$b149331/uc1.$b1...,"Abbe, George",215.292
4,uc1.32106007981415,uc1/pairtree_root/32/10/60/07/98/14/15/3210600...,"Abbey, Edward",238.725


In [146]:
sum(pd.isnull(subpaths.tokensperpage))

0

In [148]:
subpaths.to_csv('../getEF/cohort3_pathlist.tsv', sep = '\t', index = False)

In [149]:
np.median(subpaths.tokensperpage)

185.76

In [160]:
c4 = pd.read_csv('../metadata/corpus4.tsv', sep = '\t', low_memory = False)

In [161]:
len(alreadyremoved)

68

In [162]:
alreadyremoved.intersection(set(c4.docid.tolist()))

{'nyp.33433066630348'}